In [1]:
from dotenv import load_dotenv

load_dotenv()

True

### Input & Output State

In [2]:
from langchain_openai import ChatOpenAI
from langgraph.graph import END, START, StateGraph

model = ChatOpenAI()

In [3]:
from typing import TypedDict


class ChatMessages(TypedDict):
    question: str
    answer: str
    llm_calls: int

In [4]:
def call_model(state: ChatMessages):
    question = state["question"]
    llm_calls = state.get("llm_calls", 0)
    state["llm_calls"] = llm_calls + 1
    print("LLM_CALLS:", state["llm_calls"])
    response = model.invoke(input=question)
    state["answer"] = response.content
    return state

In [5]:
workflow = StateGraph(ChatMessages)

workflow.add_edge(START, "agent")
workflow.add_node("agent", call_model)
workflow.add_edge("agent", END)

graph = workflow.compile()

In [6]:
graph.invoke(input={"question": "Whats the highest mountain in the world?"})

LLM_CALLS: 1


{'question': 'Whats the highest mountain in the world?',
 'answer': 'Mount Everest, which stands at 29,029 feet (8,848 meters) above sea level, is the highest mountain in the world.',
 'llm_calls': 1}

In [7]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict


class InputState(TypedDict):
    question: str


class PrivateState(TypedDict):
    llm_calls: int


class OutputState(TypedDict):
    answer: str


class OverallState(InputState, PrivateState, OutputState):
    pass

In [8]:
workflow = StateGraph(OverallState, input=InputState, output=OutputState)

workflow.add_edge(START, "agent")
workflow.add_node("agent", call_model)
workflow.add_edge("agent", END)

graph = workflow.compile()

C:\Users\mrcra\AppData\Local\Temp\ipykernel_8184\902030229.py:1: LangGraphDeprecatedSinceV05: `input` is deprecated and will be removed. Please use `input_schema` instead. Deprecated in LangGraph V0.5 to be removed in V2.0.
  workflow = StateGraph(OverallState, input=InputState, output=OutputState)
C:\Users\mrcra\AppData\Local\Temp\ipykernel_8184\902030229.py:1: LangGraphDeprecatedSinceV05: `output` is deprecated and will be removed. Please use `output_schema` instead. Deprecated in LangGraph V0.5 to be removed in V2.0.
  workflow = StateGraph(OverallState, input=InputState, output=OutputState)


In [9]:
graph.invoke({"question": "Whats the highest mountain in the world?"})

LLM_CALLS: 1


{'answer': 'Mount Everest'}

### Add runtime configuration

In [10]:
from langgraph.graph import StateGraph, START, END
from langchain_core.runnables.config import RunnableConfig
from langchain.schema import SystemMessage, HumanMessage


def call_model(state: OverallState, config: RunnableConfig):
    language = config["configurable"].get("language", "English")
    system_message_content = "Respond in {}".format(language)
    system_message = SystemMessage(content=system_message_content)
    messages = [system_message, HumanMessage(content=state["question"])]
    response = model.invoke(messages)
    return {"answer": response}

In [11]:
workflow = StateGraph(ChatMessages)

workflow.add_edge(START, "agent")
workflow.add_node("agent", call_model)
workflow.add_edge("agent", END)

graph = workflow.compile()

In [12]:
config = {"configurable": {"language": "Spanish"}}
graph.invoke({"question": "What's the highest mountain in the world?"}, config=config)

{'question': "What's the highest mountain in the world?",
 'answer': AIMessage(content='La montaña más alta del mundo es el Monte Everest, con una altitud de 8,848 metros sobre el nivel del mar. Se encuentra en la cordillera del Himalaya, en la frontera entre Nepal y China.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 23, 'total_tokens': 71, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Bp99ZDoYTaUzBrEqaLKLgOWo5mZ57', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--da87e7d5-80f0-4cde-aada-b8de78b441b8-0', usage_metadata={'input_tokens': 23, 'output_tokens': 48, 'total_tokens': 71, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_d

In [13]:
config = {"configurable": {"language": "German"}}
graph.invoke({"question": "What's the highest mountain in the world?"}, config=config)

{'question': "What's the highest mountain in the world?",
 'answer': AIMessage(content='Der höchste Berg der Welt ist der Mount Everest in den Himalaya-Bergen.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 23, 'total_tokens': 42, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Bp99aglIDJWKgcrAmlrAGwNyDCSah', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--a69c4eee-8767-454e-b403-fe3adbbcbd2e-0', usage_metadata={'input_tokens': 23, 'output_tokens': 19, 'total_tokens': 42, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})}